# Analyse des données de Parking (SAE15)
Analyse des flux de stationnement (Voitures vs Vélos) à Montpellier.
Données issues de l'API Montpellier 3M (stockées en SQLite).

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration visuelle
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Connexion à la BDD
db_path = "parking_data.db"
conn = sqlite3.connect(db_path)

# Chargement des données
df_car = pd.read_sql_query("SELECT * FROM car_parking", conn)
df_bike = pd.read_sql_query("SELECT * FROM bike_parking", conn)

conn.close()

print(f"Données Voitures chargées : {df_car.shape}")
print(f"Données Vélos chargées : {df_bike.shape}")

In [ ]:
# Conversion Datetime
df_car['timestamp'] = pd.to_datetime(df_car['timestamp'])
df_bike['timestamp'] = pd.to_datetime(df_bike['timestamp'])

# Nettoyage (Exclure les totaux nuls pour éviter division par zéro)
df_car = df_car[df_car['totalSpotNumber'] > 0].copy()
df_bike = df_bike[df_bike['totalSlotNumber'] > 0].copy()

# --- CALCULE DES INDICATEURS ---
# Voiture : Occupation = (Total - Disponible) / Total
df_car['taux_occupation'] = ((df_car['totalSpotNumber'] - df_car['availableSpotNumber']) / df_car['totalSpotNumber']) * 100

# Vélo : Occupation = Available (présents) / Total Slot
# Hypothèse : 'availableBikeNumber' correspond au nombre de vélos présents dans la station
df_bike['taux_occupation'] = (df_bike['availableBikeNumber'] / df_bike['totalSlotNumber']) * 100

# Filtrer Semaine (Lundi=0 ... Dimanche=6) -> Garder Lundi-Vendredi (< 5)
df_car = df_car[df_car['timestamp'].dt.dayofweek < 5]
df_bike = df_bike[df_bike['timestamp'].dt.dayofweek < 5]

print("Aperçu Voitures calculees:")
display(df_car[['timestamp', 'name', 'taux_occupation']].head(3))

## 1. Profil Journalier Moyen (Heure par Heure)
Nous allons observer comment le taux d'occupation évolue au fil de la journée (Moyenne par heure).

In [ ]:
# Agrégation par Heure
df_car['heure'] = df_car['timestamp'].dt.hour
df_bike['heure'] = df_bike['timestamp'].dt.hour

profil_car = df_car.groupby('heure')['taux_occupation'].mean()
profil_bike = df_bike.groupby('heure')['taux_occupation'].mean()

# Graphique 1 : Profil Journalier
plt.figure(figsize=(12, 6))
plt.plot(profil_car.index, profil_car.values, label='Voitures (Parkings)', marker='o', linewidth=2.5, color='tab:blue')
plt.plot(profil_bike.index, profil_bike.values, label='Vélos (Stations)', marker='s', linewidth=2.5, linestyle='--', color='tab:green')

plt.title("Évolution moyenne du taux d'occupation en semaine", fontsize=16)
plt.xlabel("Heure de la journée (0-23h)", fontsize=12)
plt.ylabel("Taux d'occupation (%)", fontsize=12)
plt.xticks(range(0, 24))
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Analyse de Corrélation
Y a-t-il un lien entre l'occupation des parkings voitures et celle des stations vélos au niveau global ?
Nous allons croiser les données moyennées par heure pour voir si une tendance se dégage.

In [ ]:
# Rééchantillonnage horaire (Global)
# On fait la moyenne de TOUS les capteurs pour chaque heure disponible
data_car_h = df_car.set_index('timestamp').resample('h')['taux_occupation'].mean()
data_bike_h = df_bike.set_index('timestamp').resample('h')['taux_occupation'].mean()

# Création DataFrame commun (alignement des timestamps)
df_corr = pd.DataFrame({
    'Occupation_Car': data_car_h,
    'Occupation_Bike': data_bike_h
}).dropna()

# Calcul de la Corrélation
corr = df_corr['Occupation_Car'].corr(df_corr['Occupation_Bike'])

# Graphique 2 : Scatter Plot
plt.figure(figsize=(8, 8))
sns.regplot(x='Occupation_Car', y='Occupation_Bike', data=df_corr, 
            scatter_kws={'s': 60, 'alpha': 0.6}, line_kws={'color': 'red'})

plt.title(f"Corrélation Voiture vs Vélo (Pearson: {corr:.2f})", fontsize=15)
plt.xlabel("Taux Occupation Voiture (%)")
plt.ylabel("Taux Occupation Vélo (%)")
plt.grid(True, linestyle='--')
plt.show()

print(f"Coefficient de corrélation : {corr:.4f}")

## Conclusion
Cette analyse porte sur les flux de **semaine (Lundi-Vendredi)**.

- Le **Graphique 1** permet d'identifier les pics d'affluence (matin/soir) et de comparer les horaires de saturation.
- Le **Graphique 2** indique la nature de la relation. Si la corrélation est forte, cela suggère une synchronicité des usages (ex: tout le monde se gare en même temps). Si elle est négative, cela pourrait indiquer un report modal ou des usages inversés.

**Note :** Pour une analyse plus fine, il faudrait distinguer les jours de week-end où les dynamiques (loisirs) sont probablement différentes.